# PePR: Foolbox Attacks
Attack a single target with a foolbox attack.

## Prepare Environment
**Important: Restart the Runtime after this Cell!**
The restart is needed because of `pip install -e`.

In [ ]:
!git clone https://github.com/hallojs/ml-pepr.git
%pip install ./ml-pepr
%pip install pylatex

## Imports

In [1]:
from pepr.robustness import foolbox_wrapper

import tensorflow as tf

import numpy as np
import logging

## Setup Logging

In [2]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', '%Y-%m-%d %H:%M:%S')

# TensorFlow Logger
file_handler_tf = logging.FileHandler('tf.log')
file_handler_tf.setLevel(logging.INFO)
file_handler_tf.setFormatter(formatter)

tf.get_logger().setLevel(logging.INFO)
logger_tf = tf.get_logger()
logger_tf.addHandler(file_handler_tf)

# PePR Logger
file_handler_pr = logging.FileHandler('pepr.robustness.foolbox_wrapper.log')
file_handler_pr.setLevel(logging.DEBUG)
file_handler_pr.setFormatter(formatter)

stream_handler_pr = logging.StreamHandler()
stream_handler_pr.setLevel(logging.DEBUG)
stream_handler_pr.setFormatter(formatter)

logger_pr = logging.getLogger('pepr.robustness.foolbox_wrapper')
logger_pr.addHandler(file_handler_pr)
logger_pr.addHandler(stream_handler_pr)

## Functions
Functions for creating models and preparing the dataset.

In [3]:
def get_target_model(input_shape, number_of_labels):
    target_model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation="tanh", padding='same', input_shape=input_shape),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation="tanh", padding='same'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="tanh"),
        tf.keras.layers.Dense(number_of_labels),
        tf.keras.layers.Softmax()
    ])
    return target_model

def create_compile_model():
    from tensorflow.keras import models
    from tensorflow.keras import optimizers
    input_shape = (28, 28, 1)
    number_classes = 10

    model = get_target_model(input_shape, number_classes)

    optimizer = optimizers.Adam(lr=0.0001)
    loss = 'sparse_categorical_crossentropy'
    metrics = ["accuracy"]
    model.compile(optimizer, loss=loss, metrics=metrics)

    return model

def load_fashion_mnist():
    train, test = tf.keras.datasets.fashion_mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    # Normalize the data to a range between 0 and 1
    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    # Reshape the images to (28, 28, 1)
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    return np.vstack((train_data, test_data)), np.hstack((train_labels, test_labels))

## Init Dataset

In [ ]:
data, labels = load_fashion_mnist()

## Train Target Model

In [ ]:
target_model = create_compile_model()
target_model.fit(data[40000:50000],
                 labels[40000:50000],
                 epochs=10,
                 batch_size=50,
                 verbose=1)
target_model.save('data/target_model')

## Run the Attack
Run the [LinfDeepFoolAttack](https://hallojs.github.io/ml-pepr/foolbox_wrapper.html#pepr.robustness.foolbox_wrapper.LinfDeepFoolAttack) attack with one optional parameter `steps`.

In [ ]:
attack_pars = {
    "epsilons": np.linspace(0.1,1,10),
    "steps": 50,
}

data_conf = {
    "attack_indices_per_target": np.array([np.arange(100)])
}

attack = foolbox_wrapper.LinfDeepFoolAttack("DeepFool", attack_pars, data, labels, data_conf, [target_model])

attack.run()

### Create Attack Report

In [8]:
attack.create_attack_report()

In [9]:
# Zip report directory if you want to download it from google colab
!zip -r -q report.zip foolbox_report